# Analyse news from snopes



In [4]:
#Imports
import json
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import lxml
from urllib.request import Request, urlopen

In [6]:
 ### Recolher os links que se pretende analisar
links = []
for i in range (1,150):

    req = Request("https://www.snopes.com/fact-check/page/"+str(i))
    html_page = urlopen(req)

    soup = BeautifulSoup(html_page, "html.parser")

  
    for link in soup.findAll('a'):
        if type (link.get('href')) == str and (len( link.get('href'))) >  50 and link.get('href').startswith('https://www.snopes.com/fact-check/'):  
            links.append(link.get('href'))


In [9]:
list_urls = links[:-1]
list_urls

['https://www.snopes.com/fact-check/pledge-of-allegiance-ban/',
 'https://www.snopes.com/fact-check/texas-homeowner-robbery-ak47/',
 'https://www.snopes.com/fact-check/video-7000-fireworks-san-diego-2012/',
 'https://www.snopes.com/fact-check/waterpark-slide-launch-people-into-air/',
 'https://www.snopes.com/fact-check/secret-service-debunked-hutchinson/',
 'https://www.snopes.com/fact-check/clarence-thomas-cells-aborted/',
 'https://www.snopes.com/fact-check/marines-jetpacks-video/',
 'https://www.snopes.com/fact-check/rudy-giuliani-sells-mypillow-sandals/',
 'https://www.snopes.com/fact-check/alan-turing-facebook-post/',
 'https://www.snopes.com/fact-check/trump-ukraine-quote/',
 'https://www.snopes.com/fact-check/uterus-mailed-supreme-court/',
 'https://www.snopes.com/fact-check/jodie-sweetin-police-abortion-protest/',
 'https://www.snopes.com/fact-check/lauren-boebert-two-near-naked-men/',
 'https://www.snopes.com/fact-check/hotel-luggage-bathtub/',
 'https://www.snopes.com/fact-ch

In [16]:
language = 'en'

def extractor (url):
    '''
    
        1) Extrair conteudo do url da noticia
        2) Identificar as componentes na noticia: alegaçao, avalicao, editor, url
        3) Guardar em dataset
        
        
    '''
  
    new = requests.get(url)
    soup = BeautifulSoup(new.content, 'html.parser')

    
    #Extract Title
    title = soup.find(class_='title').contents[0].strip('\n').strip('\t')

    #extract description
    for tag in soup.find_all("meta"):
        if tag.get("name", None) == "description":
            description = tag.get("content", None)
            
     #extract claim
    claim = soup.find(class_="claim-text card-body").contents[0].strip('\n').strip('\t') 
            
            
    #extract classification         
    evaluation =  soup.find(class_='media-body d-flex flex-column align-self-center')
    for span in evaluation.find_all('span', recursive=False):
        classification = span.get("class", None)[1].replace("rating-label-", "")
        
    editor = url.split('.')[1]

     #extract data 
    for span in soup.find_all('time'):
        datatime = span.get("datetime", None)
       
    
    #  text = readNews.main(url, "text").encode('utf8').decode().split('\n')
    
    # Extract the text of the new
    text = []
    for data in soup.find(class_='single-body card-body rich-text').find_all("p"): 
        if data.get_text() == "Sources:":
            break
        text.append(data.get_text()) 

  
    return title , description, claim, text, classification, editor, datatime


    
def main ():
  
    for url in list_urls:
        
        '''
        Verificar se o link ainda nao foi tratado
        '''
        with open("controller.txt") as f:
            lines = [url.strip('\n') for url in f.readlines()]
            f.close()
            if url in lines:
               
                continue

        
        '''
        Verificar se o link esta no formato fact checking
        '''
        new = requests.get(url)
        soup = BeautifulSoup(new.content, 'html.parser')
    
        typeNew = soup.find_all(class_="breadcrumb-item")[1].get_text()
        
        if (typeNew.strip('\n').strip('\t') == 'Fact Checks' ):
            
            '''
            Add link to a control page
            '''
            with open("controller.txt", "a", encoding = 'utf8') as f:
                f.write(url)
                f.write('\n')
                f.close()

            title , description,  claim, text, classification, editor, date =  extractor (url)

            # Getting the current date and time
            dt = datetime.now()

            # getting the timestamp
            ts = datetime.timestamp(dt)

            obj = { 'id':str(ts),  "date": date, 'title': title, "allegation": claim, "evaluation": classification , "nameNewspaper":editor , 'url': url, "language": language , "postText": text }

            #print(obj)
            print(url, ts) 
            with open("extractions/" + editor + "_extraction_" + str(ts)+ ".json", "w", encoding = 'utf8') as jsonFile:
                json.dump(obj, jsonFile, indent = 4, ensure_ascii=False)


In [ ]:
main()